# Estimación de variables espaciales
<b>Proyecto</b>:	PDB IV - Pobreza urbana
<b>Objetivo</b>:    Desarrollo de variables espaciales y tabulaciones de pobreza con variables 

### 1. Bases de datos y librerías

In [177]:
import pandas as pd
import os
import numpy as np
import geopandas as gdp
import matplotlib.pyplot as plt

In [117]:
os.chdir(r'C:\Users\User\OneDrive - MIGRACIÓN VIDENZA\1. Proyectos\1. Proyectos actuales\23. Artículos PDB\2. PDB - Pobreza Urbana\2. Data\1. Bases\2. ENAHO Anual')

mod5 = pd.read_stata('modulo500.dta', convert_categoricals=False,
                     columns= ['anio', 'mes', 'conglome', 'vivienda','hogar', 'ubigeo', 'dominio', 'estrato',
                               'codperso', 'fac500a', 'p558d2_1', 'p558d2_2', 'ocupinf', 'i524a1', 'd529t',
                               'i530a', 'd536', 'i538a1', 'd540t', 'i541a', 'd543', 'd544t', 'p521a', 'p521',
                               'p519','i520', 'i513t', 'i518'])

sumaria = pd.read_stata('sumaria.dta', convert_categoricals=False,
                        columns= ['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'percepho', 'mieperho', 'gashog2d',
                                  'linpe', 'linea', 'gashog1d', 'pobrezav', 'pobreza', 'ipcr_0', 'gpgru0'])

dfEnaho = pd.merge(mod5, sumaria, how='inner', on=['anio', 'mes', 'conglome', 'vivienda', 'hogar'], validate="m:1")

del mod5, sumaria

C:\Users\User\AppData\Local\Temp\ipykernel_31464\1566263399.py:3: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  mod5 = pd.read_stata('modulo500.dta', convert_categoricals=False,
C:\Users\User\AppData\Local\Temp\ipykernel_31464\1566263399.py:9: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  sumaria = pd.read_stata('sumaria.dta', convert_categoricals=False,


In [193]:
# Nos quedamos solo con las observaciones urbanas
dfUrb = dfEnaho[dfEnaho['estrato'] < 6]

In [194]:
# Modificamos la variable p558d2_2 para ponerlo en formato ubigeo
dfUrb.loc[:,'p558d2_2'] = dfUrb['p558d2_2'].fillna('0')
dfUrb.loc[:,'p558d2_2'] = dfUrb['p558d2_2'].astype(int).astype(str).str.zfill(6)

C:\Users\User\AppData\Local\Temp\ipykernel_31464\830759936.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '0' '0' ... 230109.0 230109.0 230109.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfUrb.loc[:,'p558d2_2'] = dfUrb['p558d2_2'].fillna('0')


In [195]:
# Generamos la variable de pctResidencia
distResidencia = dfUrb.loc[:,['ubigeo','fac500a']].dropna()
distResidencia['provResidencia'] = (distResidencia['ubigeo'].astype(int) / 100).astype(int).astype(str).str.zfill(4)
distResidencia = distResidencia.dropna()

# Contabilizamos la cantidad de personas que residen por distrito
distResidencia = distResidencia.groupby(['provResidencia',"ubigeo"])['fac500a'].sum().reset_index()

distResidencia = distResidencia.rename(columns={'fac500a':'nPersonas'})
distResidencia['nPersonasProv'] = distResidencia.groupby('provResidencia')['nPersonas'].transform('sum')
distResidencia['pctResidencia'] = distResidencia['nPersonas'] / distResidencia['nPersonasProv']

In [196]:
# Generamos la variable ubigeo del centro de trabajo
dfUrb.loc[:,'ubigeoTrabajo'] = dfUrb.apply(lambda row: row['ubigeo'] if row['p558d2_1'] == 1 else row['p558d2_2'], axis=1)
dfUrb.loc[:,'ubigeoTrabajo'] = dfUrb['ubigeoTrabajo'].replace("000000", pd.NA).replace("999999", pd.NA)

C:\Users\User\AppData\Local\Temp\ipykernel_31464\3557774175.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfUrb.loc[:,'ubigeoTrabajo'] = dfUrb.apply(lambda row: row['ubigeo'] if row['p558d2_1'] == 1 else row['p558d2_2'], axis=1)


In [197]:
# Generamos la variable provincia del trabajo 
distTrabajo = dfUrb.loc[:,['ubigeoTrabajo','ubigeo','fac500a']].dropna()
distTrabajo['provTrabajo'] = (distTrabajo['ubigeoTrabajo'].astype(int) / 100).astype(int).astype(str).str.zfill(4)
distTrabajo = distTrabajo.dropna()

# Contabilizamos la cantidad de empleos que se generan
distTrabajo = distTrabajo.groupby(['provTrabajo',"ubigeoTrabajo"])['fac500a'].sum().reset_index()

In [198]:
distTrabajo = distTrabajo.rename(columns={'fac500a':'nTrabajos'})
distTrabajo['nTrabajosProv'] = distTrabajo.groupby('provTrabajo')['nTrabajos'].transform('sum')
distTrabajo['pctTrabajo'] = distTrabajo['nTrabajos'] / distTrabajo['nTrabajosProv']

In [199]:
# Definimos como centralidad si un distrito concentra más del 25% del trabajo de la provincia.
distTrabajo = distTrabajo.rename(columns={'ubigeoTrabajo':'ubigeo'})
distUrb = pd.merge(distTrabajo, distResidencia, how='inner', on='ubigeo')
distUrb = distUrb.loc[:, ['ubigeo', 'pctResidencia', 'pctTrabajo']]

cond1 = (distUrb['pctTrabajo'] > distUrb['pctResidencia'] + 0.01 )
distUrb['distCentral'] = 0
distUrb.loc[ cond1 ,'distCentral'] = 1

del distTrabajo, distResidencia, cond1

Uso de los mapas

In [ ]:
path_dist = "C:/Users/User/OneDrive - Universidad del Pacífico/1. Documentos/0. Bases de datos/11. Mapas Perú/Límite Distrital"

peru_dist = gdp.read_file(path_dist + "/LIMITE_DISTRITAL_2020_INEI_geogpsperu_juansuyo_931381206.shp")

In [200]:
var1 = peru_dist['UBIGEO']
var2 = peru_dist.centroid
var3 = var2.x
var4 = var2.y

centroids = pd.DataFrame({'ubigeo': var1,
                          'x_long' : var3,
                          'y_lat' : var4})
del var1, var2, var3, var4

C:\Users\User\AppData\Local\Temp\ipykernel_31464\2405986124.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  var2 = peru_dist.centroid


In [201]:
distUrb = pd.merge(distUrb, centroids, how='left', on='ubigeo', validate="1:1")

In [202]:
distUrb = distUrb.rename(columns={'ubigeo':'ubigeoTrabajo'})
dfUrb = pd.merge(dfUrb, distUrb, how='left', on='ubigeoTrabajo', validate="m:1")

cond2 = (dfUrb['pobreza'] < 3 )
dfUrb['pobre'] = 0
dfUrb.loc[ cond2 ,'pobre'] = 1
del cond2

In [172]:
tabla1 = pd.crosstab([dfUrb["anio"], dfUrb["distCentral"]], dfUrb["pobre"], values=dfUrb["fac500a"], aggfunc="sum", normalize='index')

In [ ]:
# Prueba de 
